# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

1.(a)k-fold cross-validation:dividing the data into K sub-samples, take one sample each time as verification data, and take the remaining K-1 samples as training data. After the model is built, it acts on the verification data and calculates the current error rate. Repeat this process K times to average the error rate and get an overall error rate. The overall error rate can be used to estimate the error rate of the current overall data for modeling.
  (b)hold-out method：randomly assign data points into two sets,usually called the training set and the test set respectively. Then build a model on training set and evaluate its performance on testing set.
  (c)cross-validation is commonly used for evaluation the supervised learning model's performance. Because cross-validation combines (averages) measures of fitness in prediction to derive a more accurate estimate of model prediction performance.
 
2.(a)Data augmentation
  (b)Regularization
  (c)Dropout
  (d)Early stopping

## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

1.Firstly,using the sigmoid or hyperbolic tangent functions as hidden unit activation functions can cause vanishing    gradient problem. The gradient becomes vanishingly small, preventing the weights from changing their values. 
  Secondly, it is difficult to calculate when using the sigmoid or hyperbolic tangent functions.

2.Because we try to find the best parameters in the hidden layers.

3.(a)After the backpropagation,we get the same weights between every two layers, which is likely to cause the model to fail and cannot converge. 
  (b)It has no effect on a logistic regression model. Because a logistic regression problem is already a convex problem.

## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [1]:
from utils import *
import numpy as np

In [20]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier:
    def __init__(self, weight_init_sd=0.05):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        weight_init=torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([self.weight_init_sd]))
        self.weights=(weight_init.sample(sample_shape=torch.Size([784, 10]))).view(784, 10)
        self.weights.requires_grad_()
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #inputs = torch.Tensor(batch_size, 1, 28, 28)
        
        batch_size=inputs.shape[0]
        inputs_tensor=inputs.reshape(batch_size,784)
        z = inputs_tensor @ self.weights
        a = torch.log(torch.exp(z) / (torch.sum(torch.exp(z),dim=1)).view([batch_size,1]))
        return a
    
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.weights]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #l1_norm = 0
        #for i in self.weights:
        l1_norm = torch.sum(torch.abs(self.weights))
        return l1_norm
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        #l2_norm = 0
        #for i in self.weights:
        l2_norm = torch.sum(self.weights**2)
        return l2_norm
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [21]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
model = MultinomialLogisticRegressionClassifier(weight_init_sd=0.05)
res = run_experiment(
    model,
    #optimizer=optim.SGD(model.parameters(), 1.0),
    optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0,
    l2_penalty_coef=0.0001,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.5752, Accuracy: 0.8553
Validation set:	Average loss: 0.3672, Accuracy: 0.8960

Epoch 1: training...
Train set:	Average loss: 0.3376, Accuracy: 0.9075
Validation set:	Average loss: 0.3197, Accuracy: 0.9080

Epoch 2: training...
Train set:	Average loss: 0.3088, Accuracy: 0.9140
Validation set:	Average loss: 0.3028, Accuracy: 0.9143

Epoch 3: training...
Train set:	Average loss: 0.2964, Accuracy: 0.9183
Validation set:	Average loss: 0.2935, Accuracy: 0.9178

Epoch 4: training...
Train set:	Average loss: 0.2893, Accuracy: 0.9201
Validation set:	Average loss: 0.2892, Accuracy: 0.9190

Epoch 5: training...
Train set:	Average loss: 0.2845, Accuracy: 0.9209
Validation set:	Average loss: 0.2861, Accuracy: 0.9180

Epoch 6: training...
Train set:	Average loss: 0.2810, Accuracy: 0.9225
Validation set:	Average loss: 0.2885, Accuracy: 0.9178

Epoch 7: training...
Train set:	Average loss: 0.2787, Accuracy: 0.9230
Validation set:	Average loss: 0.2831, Ac

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [16]:
# *CODE FOR PART 4.1 IN THIS CELL*

class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        super().__init__()
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        self.weight0=torch.normal(mean=0.0,std=weight_init_sd*torch.ones((784,128)))
        self.weight1=torch.normal(mean=0.0,std=weight_init_sd*torch.ones((128,64)))
        self.weight2=torch.normal(mean=0.0,std=weight_init_sd*torch.ones((64,32)))
        self.weight3=torch.normal(mean=0.0,std=weight_init_sd*torch.ones((32,10)))
        
        
        self.weight0.requires_grad_()
        self.weight1.requires_grad_()
        self.weight2.requires_grad_()
        self.weight3.requires_grad_()
    
    #for i in range(4):
            #self.weights[i].requires_grad_()
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        batch_size = inputs.shape[0] 
        #print(inputs.size())
        
        inputs_tensor=inputs.reshape(batch_size,784)
        #print(inputs_tensor)
        #print(inputs_tensor.size())
        
        #layer1
        z1 = inputs_tensor @ self.weight0
        a1 = self.activation(z1)
        
        #layer2
        z2 = a1 @ self.weight1
        a2 = self.activation(z2)
        
        #layer3
        z3 = a2 @ self.weight2
        a3 = self.activation(z3)
        
        #layer4
        z4 = a3 @ self.weight3
        a4 = torch.log(torch.exp(z4) / (torch.sum(torch.exp(z4),dim=1)).view([batch_size,1]))
        return a4
    
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.weight0,self.weight1,self.weight2,self.weight3]
     
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        L1_norm = torch.sum(torch.abs(self.weight0))+torch.sum(torch.abs(self.weight1))+torch.sum(torch.abs(self.weight2))+torch.sum(torch.abs(self.weight3))
        return L1_norm
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        L2_norm = torch.sum(self.weight0**2)+ torch.sum(self.weight1**2)+ torch.sum(self.weight2**2)+ torch.sum(self.weight3**2)
        return L2_norm
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [19]:
# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *

model = MultilayerClassifier(activation_fun='relu', weight_init_sd=0.02)
res = run_experiment(
    model,
    #optimizer=optim.SGD(model.parameters(), 0.1),
    
    optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0),
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.8),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0,
    l2_penalty_coef=0.000001,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.6017, Accuracy: 0.8076
Validation set:	Average loss: 0.3047, Accuracy: 0.9117

Epoch 1: training...
Train set:	Average loss: 0.2472, Accuracy: 0.9284
Validation set:	Average loss: 0.1783, Accuracy: 0.9440

Epoch 2: training...
Train set:	Average loss: 0.1615, Accuracy: 0.9526
Validation set:	Average loss: 0.1508, Accuracy: 0.9538

Epoch 3: training...
Train set:	Average loss: 0.1271, Accuracy: 0.9623
Validation set:	Average loss: 0.1437, Accuracy: 0.9575

Epoch 4: training...
Train set:	Average loss: 0.1048, Accuracy: 0.9692
Validation set:	Average loss: 0.1348, Accuracy: 0.9590

Epoch 5: training...
Train set:	Average loss: 0.0869, Accuracy: 0.9737
Validation set:	Average loss: 0.1137, Accuracy: 0.9647

Epoch 6: training...
Train set:	Average loss: 0.0717, Accuracy: 0.9785
Validation set:	Average loss: 0.1155, Accuracy: 0.9663

Epoch 7: training...
Train set:	Average loss: 0.0602, Accuracy: 0.9817
Validation set:	Average loss: 0.1053, Ac

3.Xavier initialization: This strategy initialize the weights from a distribution with zero mean and variance Var(W)=2/（n_in+n_out), where n_in and n_out are respectively the number of inputs and outputs of layers. By using Xavier initialization, we make sure that the weights are not too small but not too big to propagate accurately the signals. 